This notebook contains the initial experimentation code for generating the final summary of articles retrieved for a given image using the bing visual search api. This is done for the test data (first 1k samples). These summaries will be stored to provide initial context for the debating models.

In [1]:
import sys
sys.path.append("..")

import http.client, urllib.parse
import requests
import json
import os.path

BASE_URI = "https://api.bing.microsoft.com/v7.0/images/visualsearch"
SUBSCRIPTION_KEY = '***'

from utils.data import get_data
from utils.external_retrieval import get_summary, get_webpage_text

In [2]:
def get_urls(file):
    matching_urls = []
    # Construct the request headers
    headers = {
        "Ocp-Apim-Subscription-Key": SUBSCRIPTION_KEY
    }

    # Send the POST request
    response = requests.post(
        BASE_URI,
        headers=headers,
        files=file
    )

    # Check if the request was successful
    if response.status_code == 200:
        # Process the response data
        data = response.json()
        #print(data)
        result_list = []
        if len(data['tags'][0]['actions'][0]['data']['value']) == 0:
            result_list = data['tags'][0]['actions'][2]['data']['value']
        else:
            result_list = data['tags'][0]['actions'][0]['data']['value']
        for res in result_list:
            if len(matching_urls) == 3:
                break
            else:
                matching_urls.append(res['hostPageUrl'])
        return matching_urls
    else:
        # Handle the error
        return matching_urls
        #print(f"Error: {response.status_code} - {response.text}")